# Imports

In [ ]:
import os
import numpy as np 
import tqdm # version 4.40.0

import torch
from torchvision import datasets, transforms

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from cpmpy import * 

# Make/load the sudoku boards

In [ ]:
# ----- Make fixed sudoku boards for training and testing -----
# train sudoku boards consist of train MNIST images, test sudoku boards of test MNIST images

# Sudoku boards
# from https://github.com/locuslab/SATNet/blob/master/exps/sudoku.py
datadir = "sudoku"
with open(os.path.join(datadir,'features.pt'), 'rb') as f:
    X_in = torch.load(f)
with open(os.path.join(datadir,'labels.pt'), 'rb') as f:
    Y_in = torch.load(f)

# Divide the boards (2000 test, 8000 train)
#test_ids = [i for i in range(0, len(X_in), 5)]
#train_ids = [i for i in range(0, len(X_in)) if i not in test_ids]
train_ids = [i for i in range(0, 7000)]
validation_ids = [i for i in range(7000, 8000)]
test_ids = [i for i in range(8000, 10000)]

# Normalize MNIST data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the MNIST data
testset = datasets.MNIST('.', download=True, train=False, transform=transform)
digit_indices_test = {k:torch.LongTensor(*np.where(testset.targets == k)) for k in range(0,10)}
trainset = datasets.MNIST('.', download=True, train=True, transform=transform)

# Make validation set
# Initialize counters for each digit class in the validation set
count_per_class = [0] * 10
# Initialize arrays for the validation set
validation_set = []
train_set = []
# Iterate through the training set and move samples to the validation set
for img, label in trainset:
    if count_per_class[label] < 1000:
        validation_set.append([img, label])
        count_per_class[label] += 1
    else:
        train_set.append([img, label])
validation_labels = np.array([x[1] for x in validation_set])
digit_indices_validation = {k:torch.LongTensor(*np.where(validation_labels == k)) for k in range(0,10)}
train_labels = np.array([x[1] for x in train_set])
digit_indices_train = {k:torch.LongTensor(*np.where(train_labels == k)) for k in range(0,10)}


# Define constants
N_CHANNEL = 1 # MNIST images are in grey scale 
IMAGE_WIDTH = 28 # MNIST image width
IMAGE_HEIGHT = 28 # MNIST image height


# fix the seed for board generation
torch.manual_seed(9168)
np.random.seed(9168)
rng = np.random.RandomState(243)


def sample_visual_sudoku_test(sudoku_puzzle):
    """
        Turn the given `sudoku_puzzle` into a visual puzzle by replace numeric values
        by images from MNIST. 
    """
    # Visual Sudoku Tensor of shape puzzle_width x puzzle_height x n_channel x image_width x image_height
    sudoku_torch_dimension = sudoku_puzzle.shape + (N_CHANNEL, IMAGE_WIDTH, IMAGE_HEIGHT,)
    vizsudoku = torch.zeros(sudoku_torch_dimension, dtype=torch.float32)

    # sample dataset indices for each non-zero digit
    #for val in np.unique(sudoku_puzzle[sudoku_puzzle > 0]):
    for val in np.unique(sudoku_puzzle):
        val_idx = np.where(sudoku_puzzle == val)
        # randomly sample different MNIST images for a given digit all at once
        idx = torch.LongTensor(rng.choice(digit_indices_test[val], len(sudoku_puzzle[val_idx])))
        vizsudoku[val_idx] = torch.stack([testset[i][0] for i in idx])
        #print(val, idx)
    return vizsudoku


def sample_visual_sudoku_test_correlated(sudoku_puzzle):
    """
        Turn the given `sudoku_puzzle` into a visual puzzle by replace numeric values
        by images from MNIST. 
    """
    # Visual Sudoku Tensor of shape puzzle_width x puzzle_height x n_channel x image_width x image_height
    sudoku_torch_dimension = sudoku_puzzle.shape + (N_CHANNEL, IMAGE_WIDTH, IMAGE_HEIGHT,)
    vizsudoku = torch.zeros(sudoku_torch_dimension, dtype=torch.float32)

    # sample dataset indices for each non-zero digit
    #for val in np.unique(sudoku_puzzle[sudoku_puzzle > 0]):
    for val in np.unique(sudoku_puzzle):
        val_idx = np.where(sudoku_puzzle == val)
        # randomly sample different MNIST images for a given digit all at once
        idx = torch.LongTensor(rng.choice(digit_indices_test[val], len(sudoku_puzzle[val_idx])))
        # Add noise
        images_with_noise = [add_noise(testset[idx[0]][0], noise_level=0.75) for i in idx]
        vizsudoku[val_idx] = torch.stack([torch.tensor(img) for img in images_with_noise])
    return vizsudoku


def sample_visual_sudoku_train(sudoku_puzzle):
    """
        Turn the given `sudoku_puzzle` into a visual puzzle by replace numeric values
        by images from MNIST. 
    """
    # Visual Sudoku Tensor of shape puzzle_width x puzzle_height x n_channel x image_width x image_height
    sudoku_torch_dimension = sudoku_puzzle.shape + (N_CHANNEL, IMAGE_WIDTH, IMAGE_HEIGHT,)
    vizsudoku = torch.zeros(sudoku_torch_dimension, dtype=torch.float32)

    # sample dataset indices for each non-zero digit
    #for val in np.unique(sudoku_puzzle[sudoku_puzzle > 0]):
    for val in np.unique(sudoku_puzzle):
        val_idx = np.where(sudoku_puzzle == val)
        # randomly sample different MNIST images for a given digit all at once
        idx = torch.LongTensor(rng.choice(digit_indices_train[val], len(sudoku_puzzle[val_idx])))
        vizsudoku[val_idx] = torch.stack([train_set[i][0] for i in idx])
        #print(val, idx)
    return vizsudoku


def sample_visual_sudoku_validation(sudoku_puzzle):
    """
        Turn the given `sudoku_puzzle` into a visual puzzle by replace numeric values
        by images from MNIST. 
    """
    # Visual Sudoku Tensor of shape puzzle_width x puzzle_height x n_channel x image_width x image_height
    sudoku_torch_dimension = sudoku_puzzle.shape + (N_CHANNEL, IMAGE_WIDTH, IMAGE_HEIGHT,)
    vizsudoku = torch.zeros(sudoku_torch_dimension, dtype=torch.float32)

    # sample dataset indices for each non-zero digit
    #for val in np.unique(sudoku_puzzle[sudoku_puzzle > 0]):
    for val in np.unique(sudoku_puzzle):
        val_idx = np.where(sudoku_puzzle == val)
        # randomly sample different MNIST images for a given digit all at once
        idx = torch.LongTensor(rng.choice(digit_indices_validation[val], len(sudoku_puzzle[val_idx])))
        vizsudoku[val_idx] = torch.stack([validation_set[i][0] for i in idx])
        #print(val, idx)
    return vizsudoku


def sample_visual_sudoku_validation_correlated(sudoku_puzzle):
    """
        Turn the given `sudoku_puzzle` into a visual puzzle by replace numeric values
        by images from MNIST. 
    """
    # Visual Sudoku Tensor of shape puzzle_width x puzzle_height x n_channel x image_width x image_height
    sudoku_torch_dimension = sudoku_puzzle.shape + (N_CHANNEL, IMAGE_WIDTH, IMAGE_HEIGHT,)
    vizsudoku = torch.zeros(sudoku_torch_dimension, dtype=torch.float32)

    # sample dataset indices for each non-zero digit
    #for val in np.unique(sudoku_puzzle[sudoku_puzzle > 0]):
    for val in np.unique(sudoku_puzzle):
        val_idx = np.where(sudoku_puzzle == val)
        # randomly sample different MNIST images for a given digit all at once
        idx = torch.LongTensor(rng.choice(digit_indices_validation[val], len(sudoku_puzzle[val_idx])))
        # Add noise
        images_with_noise = [add_noise(validation_set[idx[0]][0], noise_level=0.75) for i in idx]
        vizsudoku[val_idx] = torch.stack([torch.tensor(img) for img in images_with_noise])
    return vizsudoku


# Define a function to add random noise to an image
def add_noise(image, noise_level=0.1):
    noise = np.random.normal(scale=noise_level, size=image.shape)
    noisy_image = image + noise
    return torch.tensor(noisy_image.clip(-1, 1), dtype=torch.float32).clone().detach()  # Ensure values are within [-1, 1] range


def transform_labels(sudoku):
    new_labels = torch.zeros(9, 9)
    tensor_digits = torch.arange(1, 10)
    for i in range(0, 9):
        for j in range(0, 9):
            digit_vector = sudoku[i, j]
            new_labels[i, j] = torch.sum(tensor_digits * digit_vector)
    return new_labels.numpy().astype(int)


# ----- Do the test sudokus -----
visual_test_sudokus = []
numerical_test_sudokus = []
completed_numerical_test_sudokus = []

for i in test_ids:
    completed_numerical_sudoku = torch.from_numpy(transform_labels(Y_in[i]))
    numerical_sudoku = torch.from_numpy(transform_labels(X_in[i]))
    visual_sudoku = sample_visual_sudoku_test(transform_labels(X_in[i]))
    visual_test_sudokus.append(visual_sudoku)
    numerical_test_sudokus.append(numerical_sudoku)
    completed_numerical_test_sudokus.append(completed_numerical_sudoku)
 
# torch.save(visual_test_sudokus, 'visual_test_sudokus.pth')
# torch.save(numerical_test_sudokus, 'numerical_test_sudokus.pth')
# torch.save(completed_numerical_test_sudokus, 'completed_numerical_test_sudokus.pth')


# ----- Do the correlated test sudokus -----
visual_correlated_test_sudokus = []

for i in test_ids:
    completed_numerical_sudoku = torch.from_numpy(transform_labels(Y_in[i]))
    numerical_sudoku = torch.from_numpy(transform_labels(X_in[i]))
    visual_sudoku = sample_visual_sudoku_test_correlated(transform_labels(X_in[i]))
    visual_correlated_test_sudokus.append(visual_sudoku)
 
 
# ----- Do the train sudokus -----
visual_train_sudokus = []
numerical_train_sudokus = []
completed_numerical_train_sudokus = []

#for i in train_ids:
double_train_ids = train_ids + train_ids + train_ids + train_ids + train_ids + train_ids
#double_train_ids = 15 * train_ids
for i in double_train_ids:
    completed_numerical_sudoku = torch.from_numpy(transform_labels(Y_in[i]))
    numerical_sudoku = torch.from_numpy(transform_labels(X_in[i]))
    visual_sudoku = sample_visual_sudoku_train(transform_labels(X_in[i]))
    visual_train_sudokus.append(visual_sudoku)
    numerical_train_sudokus.append(numerical_sudoku)
    completed_numerical_train_sudokus.append(completed_numerical_sudoku)

# torch.save(visual_train_sudokus, 'visual_train_sudokus.pth')
# torch.save(numerical_train_sudokus, 'numerical_train_sudokus.pth')
# torch.save(completed_numerical_train_sudokus, 'completed_numerical_train_sudokus.pth')


# ----- Do the validation sudokus -----
visual_validation_sudokus = []
numerical_validation_sudokus = []
completed_numerical_validation_sudokus = []

for i in validation_ids:
    completed_numerical_sudoku = torch.from_numpy(transform_labels(Y_in[i]))
    numerical_sudoku = torch.from_numpy(transform_labels(X_in[i]))
    visual_sudoku = sample_visual_sudoku_validation(transform_labels(X_in[i]))
    visual_validation_sudokus.append(visual_sudoku)
    numerical_validation_sudokus.append(numerical_sudoku)
    completed_numerical_validation_sudokus.append(completed_numerical_sudoku)

# torch.save(visual_validation_sudokus, 'visual_validation_sudokus.pth')
# torch.save(numerical_validation_sudokus, 'numerical_validation_sudokus.pth')
# torch.save(completed_numerical_validation_sudokus, 'completed_numerical_validation_sudokus.pth')


# ----- Do the correlated validation sudokus -----
visual_correlated_validation_sudokus = []

for i in validation_ids:
    completed_numerical_sudoku = torch.from_numpy(transform_labels(Y_in[i]))
    numerical_sudoku = torch.from_numpy(transform_labels(X_in[i]))
    visual_sudoku = sample_visual_sudoku_validation_correlated(transform_labels(X_in[i]))
    visual_correlated_validation_sudokus.append(visual_sudoku)
    

In [ ]:
# Let's also define a helper function to visualize the visual puzzle

N_CHANNEL = 1 # MNIST images are in grey scale 
IMAGE_WIDTH = 28 # MNIST image width
IMAGE_HEIGHT = 28 # MNIST image height

def show_grid_img(visual_sudoku, in_green=None, in_red=None, title=None):
    images =visual_sudoku.reshape(-1, IMAGE_HEIGHT, IMAGE_WIDTH)
    images = (255 * images).int()
    dim = visual_sudoku.shape[0]
    fig = plt.figure(figsize=(8,8))
    grid = ImageGrid(fig, 111, nrows_ncols=(dim,dim), axes_pad=0.03)
    
    if title:
        fig.suptitle(title, fontsize=16)

    # cells to plot in green | red
    N = len(images)
    if in_green is None:
        in_green = np.zeros(N, dtype=bool)
    if in_red is None:
        in_red = np.zeros(N, dtype=bool)
    in_green = in_green.flatten()
    in_red = in_red.flatten()
    
    for ax, index in zip(grid, range(len(images))):
        # dont display ticks
        for axis in ax.axis.values():
            axis.toggle(ticks=False, ticklabels=False)
        # color appropriately
        color = 'gray_r'
        if in_red[index]:
            color = 'autumn'
        if in_green[index]:
            color = 'summer'
        # and show
        ax.imshow(images[index].numpy().squeeze(), cmap=color)

In [ ]:
# Show some puzzles
show_grid_img(visual_train_sudokus[0])

# Pre-train CNN & SLN

## Digit level

In [ ]:
## Convolutional Neural Network for digit classification

from torch import nn 
import torch.nn.functional as F


class TinyLeNet(nn.Module):
    def __init__(self):
        super(TinyLeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 5, 1, padding=2)
        self.conv2 = nn.Conv2d(1, 2, 5, 1)
        self.fc1 = nn.Linear(5*5*2, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*2) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x





def load_classifier(clf_classname, path):
    """
        Initialize a new CNN classifier by
        loading pre-trained weights stored in `path` file
    """
    net = clf_classname()
    state_dict = torch.load(path, map_location=lambda storage, loc: storage)
    net.load_state_dict(state_dict)
    return net

def load_classifier2(clf_classname, path):
    """
        Initialize a new CNN classifier by
        loading pre-trained weights stored in `path` file
    """
    net = clf_classname()
    net_state_dict = net.state_dict()
    state_dict = torch.load(path, map_location=lambda storage, loc: storage)
    state_dict = {k: v for k, v in state_dict.items() if k in net_state_dict}
    net_state_dict.update(state_dict)
    net.load_state_dict(net_state_dict)

    # Freeze loaded weights
    for name, param in net.named_parameters():
        if name in state_dict:
            param.requires_grad = False
        else:
            param.requires_grad = True
    return net



@torch.no_grad()
def predict_proba_sudoku(neuralnet, vizsudoku):
    """
        Assign a probabilistic vector to each image of the visual puzzle
    """
    for param in neuralnet.parameters():
        device = param.device
        break  # Only need to check the device of one parameter
        
    grid_shape = vizsudoku.shape[:2]
    # reshape from 9x9x1x28x28 to 81x1x28x28 
    #pred = neuralnet(vizsudoku.flatten(0,1))
    # Don't do this since board level
    pred = neuralnet(vizsudoku.unsqueeze(0).to(device))
    pred = pred.view(9,9,10)
    pred = F.softmax(pred, dim=2)
    pred = pred.cpu()  # Move the tensor to the CPU
    # our NN return 81 probabilistic vector: an 81x10 matrix
    return pred.reshape(*grid_shape,10).detach() # reshape as 9x9x10 tensor for easier visualisation

In [ ]:
# Define a custom dataset class


# Set the seed for NumPy's random number generator
np.random.seed(1232)


class CustomCNNDataset(Dataset):
    def __init__(self, dataset):
        self.data = dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.data[idx][1]
        img = self.data[idx][0]
        return img, label

In [ ]:
# Train digit level CNN


# Seed CNN
torch.manual_seed(4287)


# Make dataloaders
cnn_trainset = CustomCNNDataset(train_set)
cnn_testset = CustomCNNDataset(validation_set)
cnn_train_loader = DataLoader(cnn_trainset, batch_size=64, shuffle=True)
cnn_test_loader = DataLoader(cnn_testset, batch_size=64, shuffle=False)


# Initialize the neural network and define the loss function and optimizer
cnn = TinyLeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.0005)
# Training loop
num_epochs = 30


for epoch in range(num_epochs):
    running_loss = 0.0
    cnn.train()
    for i, data in enumerate(cnn_train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 10 == 9:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(cnn_train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

            # Print test loss
            cnn.eval()
            correct = 0
            total = 0
            running_test_loss = 0.0
            batches = 0
            with torch.no_grad():
                for data in cnn_test_loader:
                    inputs, labels = data
                    outputs = cnn(inputs)
                    loss = criterion(outputs, labels)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    running_test_loss += loss.item()
                    batches += 1
        
            accuracy = 100 * correct / total
            print(f'Epoch [{epoch+1}/{num_epochs}] - Test Accuracy: {accuracy:.2f}% - Test Loss: {running_test_loss/batches:.4f}')

    
print('Training finished.')


In [ ]:
# Print test accuracy

torch.manual_seed(9168)
np.random.seed(9168)

cnn_testset = CustomCNNDataset(testset)
cnn_test_loader = DataLoader(cnn_testset, batch_size=64, shuffle=False)

cnn.eval()
correct = 0
total = 0
running_test_loss = 0.0
batches = 0
with torch.no_grad():
    for data in cnn_test_loader:
        inputs, labels = data
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_test_loss += loss.item()
        batches += 1

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}% - Test Loss: {running_test_loss/batches:.4f}')

In [ ]:
# Test accuracy of CNN on digits of the visual sudoku boards

total = 0
cnn_correct = 0
for i in range(0, 2000):
    numerical_sudoku = numerical_test_sudokus[i]
    visual_sudoku = visual_test_sudokus[i]

    for param in cnn.parameters():
        device = param.device
        break  # Only need to check the device of one parameter
    grid_shape = visual_sudoku.shape[:2]
    # reshape from 9x9x1x28x28 to 81x1x28x28 
    pred = cnn(visual_sudoku.flatten(0,1))
    #pred = cnn(visual_sudoku.unsqueeze(0).to(device))
    pred = pred.view(9,9,10)
    pred = F.softmax(pred, dim=2)
    pred = pred.cpu()  # Move the tensor to the CPU
    # our NN return 81 probabilistic vector: an 81x10 matrix
    ml_predictions =  pred.reshape(*grid_shape,10).detach() # reshape as 9x9x10 tensor for easier visualisation
    
    #ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)

    _, cnn_prediction = torch.max(torch.tensor(ml_predictions).data, -1)
    cnn_prediction = cnn_prediction.numpy()
    non_zero = (numerical_sudoku != 0).sum().item()
    cnn_correct += ((cnn_prediction == numerical_sudoku.numpy())).sum().item()
    
    total += 81
print(i, cnn_correct/total)

In [ ]:
# Manual saving

torch.save(cnn.state_dict(), 'TinyLeNet_0.9.pth')

In [ ]:
# Manual loading

cnn = LeNet()
cnn.load_state_dict(torch.load('LeNet_0.944.pth'))
cnn.eval()

# Count the number of weights
num_weights = sum(p.numel() for p in cnn.parameters() if p.requires_grad)
print("Number of weights in the model:", num_weights)

## Board level

In [ ]:
# Convolutional Neural Network for board classification

import torch
import torch.nn as nn
import torch.nn.functional as F



class PieceByPieceBoardCNN10(nn.Module):
    def __init__(self, use_softmax=False):
        super(PieceByPieceBoardCNN10, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, 1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 80)
        self.fc3 = nn.Linear(80, 10)
        self.fc4 = nn.Linear(81*10, 900)
        self.fc5 = nn.Linear(900, 900)
        #self.fc6 = nn.Linear(900, 900)
        self.fc7 = nn.Linear(900, 810)

        self.use_softmax = use_softmax

    def piece_by_piece(self, batch):
        # Expect shape (batchx9x9x1x28x28)
        # Turn into shape (batchx81x120) and then (batchx9720)
        device = next(self.parameters()).device
        return_tensor = torch.Tensor([]).to(device)
        batch = batch.to(device)
        for x in batch:
            # First turn 9x9x1x28x28 into 81x1x28x28 
            x = x.flatten(0,1)
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2, 2)
            x = x.view(-1, 5*5*16) 
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            return_tensor = torch.cat((return_tensor, x), 0)
        return return_tensor.view(-1, 81*10)

    def forward(self, x):
        x = self.piece_by_piece(x)
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        #x = F.relu(self.fc6(x))
        x = self.fc7(x)
        if self.use_softmax:
            x = x.view(-1, 9, 9, 10)
            x = torch.nn.functional.softmax(x, dim=len(x.shape)-1)
            x = x.view(-1, 810)
        return x

In [ ]:
# Make dataloaders

def reshape_image_tensor(input_tensor):
    output_tensor = torch.zeros(1, 252, 252)
    # Loop through the original tensor and fill the new tensor
    for i in range(9):
        for j in range(9):
            start_row = i * 28
            start_col = j * 28
            end_row = start_row + 28
            end_col = start_col + 28
            output_tensor[:, start_row:end_row, start_col:end_col] = input_tensor[i, j, 0, :, :]
    return output_tensor

# Define a custom dataset class
class CustomSudokuDataset(Dataset):
    def __init__(self, image_dataset, label_dataset):
        self.image_data = image_dataset
        self.label_data = label_dataset

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        img = reshape_image_tensor(self.image_data[idx])
        label = (self.label_data[idx]).view(-1)
        return img, label

# Define a custom dataset class
class CustomSudokuDatasetPieceByPiece(Dataset):
    def __init__(self, image_dataset, label_dataset):
        self.image_data = image_dataset
        self.label_data = label_dataset

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        img = self.image_data[idx]
        label = (self.label_data[idx]).view(-1)
        return img, label

In [ ]:
# Custom loss

def entropy(logits):
    # First softmax
    probabilities = torch.softmax(logits, dim=-1)
    entropy = -torch.sum(probabilities * torch.log(probabilities))
    return entropy

def MyLoss(inputs, targets, device):
    # Inputs of shape batch x 810, targets of shape batch x 9 x 9
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    total_sudoku_loss = 0
    for i in range(0, inputs.shape[0]):
       # Normal loss
       input = inputs[i]
       target = targets[i]
       loss = criterion(input.view(81, 10), target.view(81))
       total_loss += loss
        
       # Sudoku loss
       input = input.view(9, 9, 10)
       # For all rows
       entropy_sum = 0
       sudoku_loss_rows = 0
       for row in input:
           average_row = torch.mean(row, dim=0)
           entropy_sum += entropy(average_row)
       sudoku_loss_rows += -entropy_sum
       # For all columns
       entropy_sum = 0
       sudoku_loss_cols = 0
       for i in range(input.shape[1]):  # Iterate over columns
           column = input[:, i, :]
           average_column = torch.mean(column, dim=1)
           entropy_sum += entropy(average_column)
       sudoku_loss_cols += -entropy_sum
       # For all blocks
       entropy_sum = 0
       sudoku_loss_blocks = 0
       for a in range(0, 3):
           for b in range(0, 3):
               average_block = torch.zeros_like(input[0][0]).to(device)
               for c in range(a, a+3):
                   for d in range(b, b+3):
                       average_block += input[c][d]
               average_block = (1/9) * average_block
               entropy_sum += entropy(average_block)
       sudoku_loss_blocks += -entropy_sum
       # For all cells
       sudoku_loss_cells = 0
       for a in range(0, 9):
           for b in range(0, 9):
               sudoku_loss_cells += entropy(input[a][b])
       # Add them together
       sudoku_loss = 0.1*(sudoku_loss_rows + sudoku_loss_cols + sudoku_loss_cols) + 0.1*sudoku_loss_cells
       #sudoku_loss = sudoku_loss_rows
       total_loss += sudoku_loss
       total_sudoku_loss += sudoku_loss
        
    return total_loss/inputs.shape[0], total_sudoku_loss/inputs.shape[0]
        

In [ ]:
# Custom loss

def computeSudokuLoss(input):
    # Sudoku loss for rows
    row_average = torch.mean(input, dim=0)
    sudoku_loss_rows = -torch.sum(entropy(row_average))

    # Sudoku loss for columns
    column_average = torch.mean(input, dim=1)
    sudoku_loss_cols = -torch.sum(entropy(column_average))

    # Sudoku loss for blocks
    sudoku_loss_blocks = 0
    for a in range(0, 3):
        for b in range(0, 3):
            block = input[a:a+3, b:b+3, :]
            block_average = torch.mean(block, dim=(0, 1))
            sudoku_loss_blocks += -torch.sum(entropy(block_average))

    # Sudoku loss for cells
    sudoku_loss_cells = torch.sum(entropy(input))

    # Combine losses
    sudoku_loss = 0.01 * (sudoku_loss_rows + sudoku_loss_cols + sudoku_loss_blocks) + 0.01 * sudoku_loss_cells

    return sudoku_loss


def entropy(x):
    # Assuming x is probabilities along the last dimension
    x_softmax = torch.softmax(x, dim=-1)
    return -torch.sum(x_softmax * torch.log(x_softmax + 1e-10), dim=-1)


def MyLossFast(inputs, targets, use_sudoku_loss=True):
    # Inputs of shape batch x 810, targets of shape batch x 9 x 9
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    total_sudoku_loss = 0
    for i in range(0, inputs.shape[0]):
        # Normal loss
        input = inputs[i]
        target = targets[i]
        loss = criterion(input.view(81, 10), target.view(81))
        total_loss += loss

        if use_sudoku_loss:
            # Sudoku loss
            sudoku_loss = computeSudokuLoss(input.view(9, 9, 10))
        else:
            sudoku_loss = torch.tensor(0.0)
       
        total_loss += sudoku_loss
        total_sudoku_loss += sudoku_loss
        
    return total_loss/inputs.shape[0], total_sudoku_loss/inputs.shape[0]


# Example usage
#input = torch.randn(9, 9, 10)  # Assuming input shape is (9, 9, 10)
#loss = sudoku_loss(input)
#print(loss.item())  # Convert loss tensor to scalar

# input = torch.randn(5, 9, 9, 10)
# loss, sudoku_loss = MyLossFast(input, torch.ones(5, 9, 9).to(int))
# print(loss)

In [ ]:
# Check if you can accelerate training

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    device = mps_device
    print("Device: ", device)
    #x = torch.ones(1, device=mps_device)
    #print(x)
else:
    print ("MPS device not found.")

In [ ]:
# Train the board level CNN

# Seed
torch.manual_seed(5896)
np.random.seed(5896)

# Make dataloaders
batch_size = 16

# cnn_trainset = CustomSudokuDataset(visual_train_sudokus, numerical_train_sudokus)
# cnn_testset = CustomSudokuDataset(visual_test_sudokus, numerical_test_sudokus)
# cnn_validationset = CustomSudokuDataset(visual_validation_sudokus, numerical_validation_sudokus)

cnn_trainset = CustomSudokuDatasetPieceByPiece(visual_train_sudokus, numerical_train_sudokus)
cnn_testset = CustomSudokuDatasetPieceByPiece(visual_test_sudokus, numerical_test_sudokus)
cnn_validationset = CustomSudokuDatasetPieceByPiece(visual_validation_sudokus, numerical_validation_sudokus)

cnn_train_loader = DataLoader(cnn_trainset, batch_size=batch_size, shuffle=True)
cnn_test_loader = DataLoader(cnn_testset, batch_size=1, shuffle=False)
cnn_validation_loader = DataLoader(cnn_validationset, batch_size=1, shuffle=False)

# Initialize the neural network and define the loss function and optimizer
#trained_cnn = SudokuCNN2().to(device)
trained_cnn = PieceByPieceBoardCNN10().to(device)
trained_cnn.load_state_dict(torch.load('PieceByPieceBoardCNN10_acc_0.966.pth'))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(trained_cnn.parameters(), lr=0.000001)
num_epochs = 40


# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    running_sudoku_loss = 0.0
    trained_cnn.train()
    for i, data in enumerate(cnn_train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = trained_cnn(inputs)

        if epoch >= 40:
            loss, sudoku_loss = MyLossFast(outputs, labels, use_sudoku_loss=True)
        else:
            loss, sudoku_loss = MyLossFast(outputs, labels, use_sudoku_loss=False)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_sudoku_loss += sudoku_loss.item()

        if i % 50 == 49:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(cnn_train_loader)}], Loss: {running_loss/50:.4f}, Sudoku Loss: {running_sudoku_loss/50:.4f}')
            running_loss = 0.0
            running_sudoku_loss = 0.0

        if i % 200 == 199:
          # Print test loss
          correct = 0
          total = 0
          running_test_loss = 0.0
          running_sudoku_test_loss = 0.0
          batches = 0
          with torch.no_grad():
              for data in cnn_validation_loader:
                  inputs, labels = data
                  inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
                  outputs = trained_cnn(inputs)
                  loss, sudoku_loss = MyLossFast(outputs, labels, use_sudoku_loss=False)
                  outputs = outputs.view(81, 10)
                  labels = labels.view(81)
                  # loss = criterion(outputs, labels)
                  _, predicted = torch.max(outputs.data, 1)
                  total += labels.size(0)
                  correct += (predicted == labels).sum().item()
                  running_test_loss += loss.item()
                  running_sudoku_test_loss += sudoku_loss.item()
                  batches += 1
                  if batches > 100:
                      break
          
          accuracy = 100 * correct / total
          print(f'Epoch [{epoch+1}/{num_epochs}] - Test Accuracy: {accuracy:.2f}% - Test Loss: {running_test_loss/batches:.4f} - Sudoku test Loss: {running_sudoku_test_loss/batches:.4f}')
            
      # if i==450:
      #      break
    
print('Training finished.')

In [ ]:
# Test accuracy

trained_cnn = PieceByPieceBoardCNN10().to(device)
trained_cnn.load_state_dict(torch.load('PieceByPieceBoardCNN10_acc_0.966.pth'))

correct = 0
total = 0
running_test_loss = 0.0
running_sudoku_test_loss = 0.0
batches = 0
with torch.no_grad():
  for data in cnn_test_loader:
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
      outputs = trained_cnn(inputs)
      loss, sudoku_loss = MyLossFast(outputs, labels, use_sudoku_loss=False)
      outputs = outputs.view(81, 10)
      labels = labels.view(81)
      # loss = criterion(outputs, labels)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      running_test_loss += loss.item()
      running_sudoku_test_loss += sudoku_loss.item()
      batches += 1
      # if batches > 100:
      #     break
      
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}% - Test Loss: {running_test_loss/batches:.4f} - Sudoku test Loss: {running_sudoku_test_loss/batches:.4f}')
    

In [ ]:
# Manual saving

torch.save(trained_cnn.state_dict(), 'PieceByPieceBoardCNN9_acc_0.9.pth')

In [ ]:
# Manual loading

cnn = PieceByPieceBoardCNN9()
cnn.load_state_dict(torch.load('PieceByPieceBoardCNN9_acc_0.968.pth'))
cnn.eval()

# Sudoku solving

## The model

In [ ]:
# Own model

import cpmpy as cp
from cpmpy.solvers import CPM_ortools


def get_sudoku_model(n=9):
    b = np.sqrt(n).astype(int)
    cells = cp.IntVar(1, n, shape=(n,n))

    # plain sudoku model
    m = cp.Model(
        [cp.alldifferent(row) for row in cells],
        [cp.alldifferent(col) for col in cells.T],
        [cp.alldifferent(cells[i:i + b, j:j + b])
            for i in range(0, n, b) for j in range(0, n, b)],
    )
    return {
        'model':m,
        'variables':cells
    }


def solve_sudoku(model, dvars, instance):
    # use another object for solving
    newmodel = cp.Model(model.constraints) 
    # set given clues
    newmodel += cp.all(instance[instance>0] == dvars[instance>0])
    if newmodel.solve():
        results = {
            'runtime':np.asarray(newmodel.cpm_status.runtime),
            'solution':dvars.value(),
        }
    else:
        results = {
            'solution':np.full_like(dvars.value(), np.nan)
        }
    results['status'] = np.asarray(newmodel.cpm_status.exitstatus.value)
    return results


def get_visual_sudoku_full_image_model(ml_predictions, precision=1e-5):
    # base model and decision variables 
    visual_sudoku_problem = get_sudoku_model()
    model = visual_sudoku_problem['model']
    decision_variables = visual_sudoku_problem['variables']
    # introduce a layer of 'perception' variables, as an interface
    # between the solver and the ml network
    # their domain is [0, ..., 9], with 0 acting as the 'empty' symbol
    perception_variables = cp.intvar(0, 9, shape=decision_variables.shape, name='perception')

    # convert predictions to logspace
    logprobs = np.log(np.maximum( ml_predictions, precision ))
    # cp solver requires integer values
    logprobs = np.array(logprobs / precision).astype(int)
    # switch to cpm_array for more features
    logprobs = cp.cpm_array(logprobs)
    # build the objective function over perception variables 
    objective_function = sum(logprobs[idx][v] for idx,v in np.ndenumerate(perception_variables))
    model.maximize(objective_function)
    
    # channeling constraints to link decision variables to perception variables
    # perception variable is either 'empty' or matches grid symbol
    model+= [(perception_variables != 0).implies(decision_variables == perception_variables)]
    # keep track of perception variables as well 
    visual_sudoku_problem['perception'] = perception_variables
    return visual_sudoku_problem

def solve_visual_sudoku_full_image(visual_sudoku_problem, solver_params=dict()):
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    s = CPM_ortools(model)
    if s.solve(**solver_params):
        results = {
            'solution':dvars.value(),
            'perception':pvars.value()
        }
    else:
        # in case of infeasibility, nan
        results = {
            'solution':np.full_like(dvars.value(), np.nan),
            'perception':np.full_like(dvars.value(), np.nan)
        }
    results['status'] = np.asarray(s.cpm_status.exitstatus.value)
    results['runtime'] = np.asarray(s.cpm_status.runtime)
    return results

## Maximum Likelihood

In [ ]:
def perception_leads_to_unique_solution(perception, solution):
    sudoku_mod = get_sudoku_model(n=9)
    vars = sudoku_mod["variables"]
    model = sudoku_mod["model"]
    model += ~ cp.all((vars == solution).flatten())
    is_unique = not solve_sudoku_again(model, vars, perception)
    return is_unique


def solve_sudoku_again(model, dvars, instance):
    # use another object for solving
    newmodel = cp.Model(model.constraints) 
    # set given clues
    newmodel += cp.all(instance[instance>0] == dvars[instance>0])
    return newmodel.solve()


def solve_visual_sudoku_higher_order(visual_sudoku_problem, solver_params=dict(), max_iter=10):
    #Write a loop repeating following steps:
    # while results['solution'] is not unique or iteration < max_iter:
    #   add nogood to the vizsudoku model
    #   solve again
    #   iteration += 1
    
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    results = solve_visual_sudoku_full_image(visual_sudoku_problem, solver_params)
    #print("first ", results)
    iteration = 0
    
    while (not perception_leads_to_unique_solution(pvars.value(), dvars.value())) and (iteration < max_iter):
      # add nogood to the vizsudoku model
      #model += [~cp.all((perception_variables ==  perception_variables.value())) ]
      model += ~ cp.all((pvars == results['perception']).flatten())
      results = solve_visual_sudoku_full_image(visual_sudoku_problem, solver_params)
      iteration += 1
    if iteration >= max_iter:
        print("max iter exceeded at solving for unique solution")

    return results


In [ ]:
def inference_unique_solution(visual_sudoku, cnn):
    # convolutional neural network predictions 
    ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    
    # visual sudoku cp model
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    
    # solve 
    #results = solve_visual_sudoku_full_image(visual_sudoku_problem)
    results = solve_visual_sudoku_higher_order(visual_sudoku_problem)
    return results


def inference_simple(visual_sudoku, cnn):
    # No check for unique solution
    # convolutional neural network predictions 
    ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    
    # visual sudoku cp model
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    
    # solve 
    results = solve_visual_sudoku_full_image(visual_sudoku_problem)
    return results


def top_k_inference_unique_solution(visual_sudoku, cnn, k=1):
    return_list = []
    # convolutional neural network predictions 
    ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    # visual sudoku cp model
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    for i in range(0, k):
        results = solve_visual_sudoku_higher_order(visual_sudoku_problem)
        return_list.append(results)
        model += ~ cp.all((pvars == results['perception']).flatten())
        #model += ~ cp.all((dvars == results['solution']).flatten()) # new addition
    return return_list


def top_k_inference_simple(visual_sudoku, cnn, k=1):
    return_list = []
    # convolutional neural network predictions 
    ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    # visual sudoku cp model
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    for i in range(0, k):
        results = solve_visual_sudoku_full_image(visual_sudoku_problem)
        return_list.append(results)
        model += ~ cp.all((pvars == results['perception']).flatten())
        #model += ~ cp.all((dvars == results['solution']).flatten()) # new addition
    return return_list


def top_k_inference_simple_fast(visual_sudoku, cnn, k=1, with_hint=False):
    return_list = []
    # convolutional neural network predictions 
    ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    # visual sudoku cp model
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    solver_params=dict()
    # solve
    counter = 0
    s = SolverLookup.get("ortools", model) # faster on a solver interface directly
    while s.solve():
        counter += 1
        results = {
                'solution':dvars.value(),
                'perception':pvars.value()
        }
        results['status'] = np.asarray(s.cpm_status.exitstatus.value)
        results['runtime'] = np.asarray(s.cpm_status.runtime)
        return_list.append(results)
        # check counter
        if counter==k:
            break
        # alter constraints
        s += ~ cp.all((pvars == pvars.value()).flatten())
        # give previous solution as hint
        if with_hint:
            s.solution_hint(pvars.flatten(), pvars.value().flatten())
    return return_list


def top_k_inference_simple_fast_alternative(visual_sudoku, cnn, k=1):
    return_list = []
    # convolutional neural network predictions 
    ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    # visual sudoku cp model
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    solver_params=dict()
    for i in range(0, k):
        # Solve
        s = CPM_ortools(model)
        if s.solve(**solver_params):
            results = {
                'solution':dvars.value(),
                'perception':pvars.value()
            }
        else:
            # in case of infeasibility, nan
            results = {
                'solution':np.full_like(dvars.value(), np.nan),
                'perception':np.full_like(dvars.value(), np.nan)
            }
        results['status'] = np.asarray(s.cpm_status.exitstatus.value)
        results['runtime'] = np.asarray(s.cpm_status.runtime)
        return_list.append(results)
        # Add new constraint
        model += ~ cp.all((pvars == results['perception']).flatten())
        # give previous solution as hint
        s.solution_hint(pvars.flatten(), pvars.value().flatten())
    return return_list

## Symbolic feedback

In [ ]:
import copy


def find_index_of_element_in_arrays(arrays, t):
    """
    Find the index of an element with value t across multiple NumPy arrays.

    Parameters:
        arrays (list of numpy.ndarray): List of NumPy arrays.
        t (int or float): Value to find across the arrays.

    Returns:
        index (tuple or None): Index of the element in the arrays, or None if not found.
    """
    # Check if all arrays have the same shape
    shapes = [arr.shape for arr in arrays]
    if len(set(shapes)) != 1:
        raise ValueError("All arrays must have the same shape")

    # Stack arrays along a new axis
    stacked_array = np.stack(arrays)

    # Check if t exists in all arrays
    presence = np.all(stacked_array == t, axis=0)

    # If t is present in all arrays, find its index
    if np.any(presence):
        index = np.where(presence)
        return (index[0][0], index[1][0])  # Return only the index of the first occurrence
    else:
        return None
        

def inference_with_feedback(visual_sudoku, cnn, sln, top_k_boards=5):
    # convolutional neural network predictions 
    original_ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    ml_predictions = copy.deepcopy(original_ml_predictions)
    
    # call the solver
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    board_list = []
    complete_board_list = []
    for i in range(0, top_k_boards):
        results = solve_visual_sudoku_higher_order(visual_sudoku_problem)
        board_list.append(results)
        complete_board_list.append(results)
        model += ~ cp.all((pvars == results['perception']).flatten())
        #model += ~ cp.all((dvars == results['solution']).flatten()) # new addition
    top_boards = [board["perception"] for board in board_list]

    # refine probabilities
    for i in range(0, 9):
        for j in range(0, 9):
            initial_probabilities = ml_predictions[i, j]
            partial_evidence = [board[i, j] for board in top_boards]
            if (len(set(partial_evidence)) > 1) and (len(set(partial_evidence)) < 10):
                # Construct complete input for the sln
                complete_input = torch.zeros(1, 11, 1, 28, 28)
                complete_input[0, 0] = visual_sudoku[i, j]
                for k in range(0, 10):
                    index = find_index_of_element_in_arrays(top_boards, k)
                    if index is not None:
                        complete_input[0, k+1] = visual_sudoku[index[0], index[1]]
                with torch.no_grad():
                    refined_probability = sln(complete_input)[0]
                    refined_probability = torch.nn.functional.softmax(refined_probability, dim=0).numpy()
                ml_predictions[i, j] = refined_probability

    # Max top board
    choice = 0
    top_score = 0
    new_scores = []
    old_scores = []
    for i, board in enumerate(top_boards):
        score = 1
        old_score = 1
        for row in range(0,9):
            for col in range(0,9):
                score *= ml_predictions[row, col, board[row, col]]
                old_score *= original_ml_predictions[row, col, board[row, col]]
        new_scores.append(score)
        old_scores.append(old_score)
        if score > top_score:
            top_score = score
            choice = i 
    
    return complete_board_list[choice]


def inference_with_feedback_double_cnn(visual_sudoku, cnn, sln, top_k_boards=5):
    # convolutional neural network predictions 
    original_ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    ml_predictions = copy.deepcopy(original_ml_predictions)
    
    # call the solver
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    board_list = []
    complete_board_list = []
    for i in range(0, top_k_boards):
        results = solve_visual_sudoku_higher_order(visual_sudoku_problem)
        board_list.append(results)
        complete_board_list.append(results)
        model += ~ cp.all((pvars == results['perception']).flatten())
        #model += ~ cp.all((dvars == results['solution']).flatten()) # new addition
    top_boards = [board["perception"] for board in board_list]

    # refine probabilities
    for i in range(0, 9):
        for j in range(0, 9):
            initial_probabilities = ml_predictions[i, j]
            partial_evidence = [board[i, j] for board in top_boards]
            if (len(set(partial_evidence)) > 1) and (len(set(partial_evidence)) < 10):
                with torch.no_grad():
                    refined_probability = sln(visual_sudoku[i, j])[0]
                    refined_probability = torch.nn.functional.softmax(refined_probability, dim=0)
                ml_predictions[i, j] = refined_probability

    # Max top board
    choice = 0
    top_score = 0
    new_scores = []
    old_scores = []
    for i, board in enumerate(top_boards):
        score = 1
        old_score = 1
        for row in range(0,9):
            for col in range(0,9):
                score *= ml_predictions[row, col, board[row, col]].item()
                old_score *= original_ml_predictions[row, col, board[row, col]].item()
        new_scores.append(score)
        old_scores.append(old_score)
        if score > top_score:
            top_score = score
            choice = i
    
    return complete_board_list[choice]


def inference_with_feedback_double_cnn_simple(visual_sudoku, cnn, sln, top_k_boards=5):
    # convolutional neural network predictions 
    original_ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)
    ml_predictions = copy.deepcopy(original_ml_predictions)
    
    # call the solver
    visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    board_list = []
    complete_board_list = []
    for i in range(0, top_k_boards):
        results = solve_visual_sudoku_full_image(visual_sudoku_problem)
        board_list.append(results)
        complete_board_list.append(results)
        model += ~ cp.all((pvars == results['perception']).flatten())
        #model += ~ cp.all((dvars == results['solution']).flatten()) # new addition
    top_boards = [board["perception"] for board in board_list]

    # refine probabilities
    for i in range(0, 9):
        for j in range(0, 9):
            initial_probabilities = ml_predictions[i, j]
            partial_evidence = [board[i, j] for board in top_boards]
            if (len(set(partial_evidence)) > 1) and (len(set(partial_evidence)) < 10):
                with torch.no_grad():
                    refined_probability = sln(visual_sudoku[i, j])[0]
                    refined_probability = torch.nn.functional.softmax(refined_probability, dim=0)
                ml_predictions[i, j] = refined_probability

    # Max top board
    choice = 0
    top_score = 0
    new_scores = []
    old_scores = []
    for i, board in enumerate(top_boards):
        score = 1
        old_score = 1
        for row in range(0,9):
            for col in range(0,9):
                score *= ml_predictions[row, col, board[row, col]].item()
                old_score *= original_ml_predictions[row, col, board[row, col]].item()
        new_scores.append(score)
        old_scores.append(old_score)
        if score > top_score:
            top_score = score
            choice = i
    
    return complete_board_list[choice]

# Finetune SLN

In [ ]:
# Make train list

trained_cnn = PieceByPieceBoardCNN10()
trained_cnn.load_state_dict(torch.load('PieceByPieceBoardCNN10_SPOP_acc_0.966_2.pth'))
trained_cnn.eval()

train_list = []

for index in range(0, 3000):
    numerical_sudoku = numerical_train_sudokus[index].numpy()
    visual_sudoku = visual_train_sudokus[index]
    
    top_boards = top_k_inference_simple(visual_sudoku, trained_cnn, k=3)
    diff = (top_boards[0]["perception"]-top_boards[1]["perception"]) + (top_boards[0]["perception"]-top_boards[2]["perception"]) + (top_boards[1]["perception"]-top_boards[2]["perception"])
    true_indices = np.where(diff!=0)

    first_board = [top_boards[0]["perception"][true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]
    second_board = [top_boards[1]["perception"][true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]
    third_board = [top_boards[2]["perception"][true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]
    correct_board = [numerical_sudoku[true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]

    for k in range(0, len(first_board)):
        element = [first_board[k], second_board[k], third_board[k], correct_board[k], index, true_indices[0][k], true_indices[1][k]]
        train_list.append(element)

    if index % 20 == 19:
        print(f'Board {index}')

In [ ]:
# Make validation list

trained_cnn = PieceByPieceBoardCNN10()
trained_cnn.load_state_dict(torch.load('PieceByPieceBoardCNN10_SPOP_acc_0.966_2.pth'))
trained_cnn.eval()

validation_list = []

for index in range(0, 500):
    numerical_sudoku = numerical_validation_sudokus[index].numpy()
    visual_sudoku = visual_validation_sudokus[index]
    
    top_boards = top_k_inference_simple(visual_sudoku, trained_cnn, k=3)
    diff = (top_boards[0]["perception"]-top_boards[1]["perception"]) + (top_boards[0]["perception"]-top_boards[2]["perception"]) + (top_boards[1]["perception"]-top_boards[2]["perception"])
    true_indices = np.where(diff!=0)

    first_board = [top_boards[0]["perception"][true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]
    second_board = [top_boards[1]["perception"][true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]
    third_board = [top_boards[2]["perception"][true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]
    correct_board = [numerical_sudoku[true_indices[0][i], true_indices[1][i]] for i in range(0, len(true_indices[0]))]

    board_elements = []
    for k in range(0, len(first_board)):
        element = [first_board[k], second_board[k], third_board[k], correct_board[k], index, true_indices[0][k], true_indices[1][k]]
        board_elements.append(element)
    validation_list.append(board_elements)

    if index % 20 == 19:
        print(f'Board {index}')

In [ ]:
# Function to calculate board accuracy when using the SLN and symbolic feedback on validation boards,
# validation list has been computed earlier with the CNN

def print_acc_on_validation_boards(sln, validation_list):
    correct = 0
    total_digits = 0
    loss = 0
    criterion = nn.CrossEntropyLoss()
    
    for boards in validation_list:
        correct_prob = 1.0
        prob1 = 1.0
        prob2 = 1.0
        prob3 = 1.0
        for k in range(0, len(boards)):
            element = [boards[k][0], boards[k][1], boards[k][2], boards[k][3], boards[k][4], boards[k][5], boards[k][6]]
            total_digits += 1
            
            index = element[4]
            visual_sudoku = visual_validation_sudokus[index]
            
            output = sln(visual_sudoku[element[5], element[6]])[0]
            pred = torch.nn.functional.softmax(output, dim=0).detach().numpy()
            loss += criterion(output.unsqueeze(0), torch.tensor([element[3]]).long())
    
            correct_prob *= pred[element[3]]
            prob1 *= pred[element[0]]
            prob2 *= pred[element[1]]
            prob3 *= pred[element[2]]
            
        if np.max([prob1, prob2, prob3]) == correct_prob:
            correct += 1

    print("##### Results validation #####")
    print("Accuracy: ", correct/len(validation_list))
    print("Loss: ", loss.item()/len(validation_list))

In [ ]:
# Finetune SLN using the train list

trained_sln = TinyLeNet()
trained_sln.load_state_dict(torch.load('TinyLeNet_0.959.pth'))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(trained_sln.parameters(), lr=0.0001)
running_loss = 0

for i, element in enumerate(train_list, 0):
    index = element[4]
    numerical_sudoku = numerical_train_sudokus[index].numpy()
    visual_sudoku = visual_train_sudokus[index]
    
    # Train
    optimizer.zero_grad()
    output = trained_sln(visual_sudoku[element[5], element[6]])[0]
    label = torch.Tensor([element[3]]).long()
    loss = criterion(output.unsqueeze(0), label)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    if i % 100 == 99:
        print("------------------------------")
        print(f'Board {i}, Train Loss: {running_loss/100:.4f}')
        running_loss = 0.0
        print_acc_on_validation_boards(trained_sln, validation_list)

In [ ]:
# Validation

# Manual loading
sln = TinyLeNet()
sln.load_state_dict(torch.load('TinyLeNet_0.959.pth'))

print_acc_on_validation_boards(sln, validation_list)

In [ ]:
# Manual saving

torch.save(trained_sln.state_dict(), 'TinyLeNet_SLN.pth')

# DFL training

In [ ]:
# Define optModel

import pyepo
from pyepo.model.opt import optModel
import cpmpy as cp
from cpmpy.solvers import CPM_ortools
import copy


def mlPredictionsToLogprobs(ml_predictions, precision=1e-5):
    ml_predictions = torch.tensor(ml_predictions)
    # Convert predictions to logspace using torch
    logprobs = torch.log(torch.maximum(ml_predictions, torch.tensor(precision, dtype=ml_predictions.dtype)))
    # cp solver requires integer values
    logprobs = (logprobs / precision).to(torch.int32)
    #logprobs = (logprobs / precision)
    return logprobs
    

def transformToOneHot(input_array):
    # Convert the input array to a tensor
    input_tensor = torch.tensor(input_array)
    # Use one_hot function to create one-hot encoded tensor
    one_hot_array = torch.nn.functional.one_hot(input_tensor, num_classes=10)
    # Convert the one-hot encoded tensor to integer type
    #one_hot_array = one_hot_array.to(torch.int)
    one_hot_array = one_hot_array.to(torch.float32)
    return one_hot_array


class sudokuModel(optModel):

    def __init__(self):
        """
        Args:
            grid (tuple): size of grid network
        """
        super().__init__()
        self.logprobs = None
        self.modelSense = -1   # Maximize objective function?

    def _getModel(self):
        """
        A method to build model

        Returns:
            tuple: optimization model and variables
        """
        n = 9
        b = np.sqrt(n).astype(int)
        cells = cp.IntVar(1, n, shape=(n,n))
    
        # plain sudoku model
        m = cp.Model(
            [cp.alldifferent(row) for row in cells],
            [cp.alldifferent(col) for col in cells.T],
            [cp.alldifferent(cells[i:i + b, j:j + b])
                for i in range(0, n, b) for j in range(0, n, b)],
        )
        return_model = {
            'model':m,
            'variables':cells
        }
        return return_model, cells

    def _getFreshModel(self):
        n = 9
        b = np.sqrt(n).astype(int)
        cells = cp.IntVar(1, n, shape=(n,n))
        # plain sudoku model
        m = cp.Model(
            [cp.alldifferent(row) for row in cells],
            [cp.alldifferent(col) for col in cells.T],
            [cp.alldifferent(cells[i:i + b, j:j + b])
                for i in range(0, n, b) for j in range(0, n, b)],
        )
        fresh_model = {
            'model':m,
            'variables':cells
        }
        return fresh_model

    def setObj(self, c):
        """
        A method to set objective function

        Args:
            c : lobprobs as a numpy array
        """
        # print("  c   ")
        # print(c)
        self.probs = c.reshape(9, 9, 10)
        self.logprobs = mlPredictionsToLogprobs(self.probs).numpy()
        
    def solve(self):
        """
        A method to solve model, call strictly after calling setObj

        Returns:
            tuple: optimal solution (list) and objective value (float)
        """
        # Obtain a fresh model
        visual_sudoku_problem = self._getFreshModel()
        model = visual_sudoku_problem['model']
        decision_variables = visual_sudoku_problem['variables']
        # introduce a layer of 'perception' variables, as an interface
        # between the solver and the ml network
        # their domain is [0, ..., 9], with 0 acting as the 'empty' symbol
        perception_variables = cp.intvar(0, 9, shape=decision_variables.shape, name='perception')
        # Use the latest logprobs
        logprobs = cp.cpm_array(self.logprobs)
        visual_sudoku_problem['logprobs'] = logprobs
        # build the objective function over perception variables 
        objective_function = sum(logprobs[idx][v] for idx,v in np.ndenumerate(perception_variables))
        model.maximize(objective_function)
        # channeling constraints to link decision variables to perception variables
        # perception variable is either 'empty' or matches grid symbol
        model+= [(perception_variables != 0).implies(decision_variables == perception_variables)]
        # keep track of perception variables as well 
        visual_sudoku_problem['perception'] = perception_variables

        # Solve
        solver_params=dict()
        dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
        s = CPM_ortools(model)
        if s.solve(**solver_params):
            results = {
                'solution':dvars.value(),
                'perception':pvars.value()
            }
        else:
            # in case of infeasibility, nan
            results = {
                'solution':np.full_like(dvars.value(), np.nan),
                'perception':np.full_like(dvars.value(), np.nan)
            }
        results['status'] = np.asarray(s.cpm_status.exitstatus.value)
        results['runtime'] = np.asarray(s.cpm_status.runtime)

        # Compute value objective function
        #logprobs = visual_sudoku_problem['logprobs']
        if results['perception'][0][0] is not None:
            obj = sum(self.probs[idx][v] for idx,v in np.ndenumerate(pvars.value()))
        else:
            # In case of no solution found
            raise Exception("No solution found for the sudoku!")
        
        return transformToOneHot(results['perception']).view(810), torch.tensor(obj)



class sudokuModelWithSymbolicFeedback(optModel):

    def __init__(self, sln):
        """
        Args:
            grid (tuple): size of grid network
        """
        super().__init__()
        self.logprobs = None
        self.modelSense = -1   # Maximize objective function?
        self.sln = sln
        self.differing_digit_indices = None
        self.sln_predictions = None

    def _getModel(self):
        """
        A method to build model

        Returns:
            tuple: optimization model and variables
        """
        n = 9
        b = np.sqrt(n).astype(int)
        cells = cp.IntVar(1, n, shape=(n,n))
    
        # plain sudoku model
        m = cp.Model(
            [cp.alldifferent(row) for row in cells],
            [cp.alldifferent(col) for col in cells.T],
            [cp.alldifferent(cells[i:i + b, j:j + b])
                for i in range(0, n, b) for j in range(0, n, b)],
        )
        return_model = {
            'model':m,
            'variables':cells
        }
        return return_model, cells

    def _getDifferingDigitIndices(self):
        if self.differing_digit_indices == None:
            raise Exception("Differing digits not yet instantiated, first solve for a visual sudoku!")
        else:
            return self.differing_digit_indices

    def _getFreshModel(self):
        n = 9
        b = np.sqrt(n).astype(int)
        cells = cp.IntVar(1, n, shape=(n,n))
        # plain sudoku model
        m = cp.Model(
            [cp.alldifferent(row) for row in cells],
            [cp.alldifferent(col) for col in cells.T],
            [cp.alldifferent(cells[i:i + b, j:j + b])
                for i in range(0, n, b) for j in range(0, n, b)],
        )
        fresh_model = {
            'model':m,
            'variables':cells
        }
        return fresh_model

    def setObj(self, c):
        """
        A method to set objective function

        Args:
            c : lobprobs as a numpy array
        """
        # print("  c   ")
        # print(c)
        self.probs = c.reshape(9, 9, 10)
        self.logprobs = mlPredictionsToLogprobs(self.probs).numpy()

    def setSLNPredictions(self, ml_predictions):
        self.sln_predictions = ml_predictions
        
    def solve(self):
        """
        A method to solve model, call strictly after calling setObj

        Returns:
            tuple: optimal solution (list) and objective value (float)
        """
        # convolutional neural network predictions 
        original_ml_predictions = self.probs
        ml_predictions = copy.deepcopy(original_ml_predictions)
        
        # call the solver
        visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
        model = visual_sudoku_problem['model']
        dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
        board_list = []
        complete_board_list = []
        top_k_boards = 3
        for i in range(0, top_k_boards):
            results = solve_visual_sudoku_full_image(visual_sudoku_problem)
            board_list.append(results)
            complete_board_list.append(results)
            model += ~ cp.all((pvars == results['perception']).flatten())
            #model += ~ cp.all((dvars == results['solution']).flatten()) # new addition
        top_boards = [board["perception"] for board in board_list]

        # check if you can use sln predictions
        if self.sln_predictions != None:
    
            # refine probabilities
            differing_digit_indices = []
            for i in range(0, 9):
                for j in range(0, 9):
                    partial_evidence = [board[i, j] for board in top_boards]
                    if len(set(partial_evidence)) > 1:
                        differing_digit_indices.append((i,j))
                        with torch.no_grad():
                            #refined_probability = self.sln(visual_sudoku[i, j])[0]
                            #refined_probability = torch.tensor(ml_predictions[i, j])
                            #refined_probability = torch.nn.functional.softmax(refined_probability, dim=0)
                            refined_probability = self.sln_predictions[i, j]
                        ml_predictions[i, j] = refined_probability
            # Set the indices
            self.differing_digit_indices = differing_digit_indices
        
            # Max top board
            choice = 0
            top_score = 0
            new_scores = []
            old_scores = []
            for i, board in enumerate(top_boards):
                score = 1
                old_score = 1
                for row in range(0,9):
                    for col in range(0,9):
                        score *= ml_predictions[row, col, board[row, col]].item()
                        old_score *= original_ml_predictions[row, col, board[row, col]].item()
                new_scores.append(score)
                old_scores.append(old_score)
                if score > top_score:
                    top_score = score
                    choice = i
            results = complete_board_list[choice]

        else:
            results = board_list[0]
        
        # Compute value objective function
        if results['perception'][0][0] is not None:
            #obj = sum(self.probs[idx][v] for idx,v in np.ndenumerate(pvars.value()))
            obj = sum(self.probs[idx][v] for idx,v in np.ndenumerate(results["perception"]))
        else:
            # In case of no solution found
            raise Exception("No solution found for the sudoku!")
        
        return transformToOneHot(results['perception']).view(810), torch.tensor(obj)

In [ ]:
# Define custom dataset


class CustomSudokuDatasetDFL(Dataset):
    def __init__(self, image_dataset, label_dataset):
        self.image_data = image_dataset
        self.label_data = label_dataset

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        # input features x
        x = self.image_data[idx]
        # true optimal solution w 
        w = transformToOneHot(self.label_data[idx]).view(810)
        # true costs c (true probs for the images)
        c = w
        # true optimal objective value z is always 81
        z = 81. * torch.ones(1)
        
        return x, c, w, z


# Make datasets
trainset = CustomSudokuDatasetDFL(visual_train_sudokus[0:20000], numerical_train_sudokus[0:20000])
validationset = CustomSudokuDatasetDFL(visual_validation_sudokus[0:10], numerical_validation_sudokus[0:10])

In [ ]:
# Check if you can accelerate training

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    device = mps_device
    print("Device: ", device)
    #x = torch.ones(1, device=mps_device)
    #print(x)
else:
    print ("MPS device not found.")

## Without SLN

In [ ]:
# Training without SLN, SPO+

# Seed
torch.manual_seed(5896)
np.random.seed(5896)

# init prediction model
predmodel = PieceByPieceBoardCNN10(use_softmax=True).to(device)
predmodel.load_state_dict(torch.load('PieceByPieceBoardCNN10_acc_0.966.pth'))
# set optimizer
optimizer = torch.optim.Adam(predmodel.parameters(), lr=1e-7)
# init optimization model
optmodel = sudokuModel()
# init SPO+ loss
spop = pyepo.func.SPOPlus(optmodel, processes=5)
regret_list = []

batch_size = 8
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False)
num_epochs = 1

# training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        # forward pass
        predictions = predmodel(x)
        #predictions = predmodel(x).view(x.shape[0],9,9,10)
        # Apply softmax to make probability distributions from logits
        #predictions = torch.nn.functional.softmax(predictions, dim=len(predictions.shape)-1)
        #predictions = predictions.view(x.shape[0],810)
        loss = spop(predictions, c, w, z, reduction="mean")
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print("Epoch {:2},  Batch: {:2},  Loss: {:}".format(epoch+1, i+1, loss.item()))
        if i % 25 == 24:
            regret = pyepo.metric.regret(predmodel, optmodel, validation_loader)
            regret_list.append(regret)
            print("Regret: ", regret)
        if i == 1:
            break


In [ ]:
# Make optDataset for Pairwise learning to rank

# Define the input features x and true costs c
x_train = visual_train_sudokus[30000:40000]
c_train = [transformToOneHot(solution).view(810) for solution in numerical_train_sudokus[30000:40000]]
# Get optDataset
dataset_train = pyepo.data.dataset.optDataset(optmodel, x_train, c_train)
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

In [ ]:
# Training without SLN, Pairwise learning to rank

# Seed
torch.manual_seed(5896)
np.random.seed(5896)

# init prediction model
#predmodel = PieceByPieceBoardCNN10(use_softmax=True).to(device)
#predmodel.load_state_dict(torch.load('PieceByPieceBoardCNN10_pairwise_acc_0.966_2.pth'))
# set optimizer
optimizer = torch.optim.Adam(predmodel.parameters(), lr=1e-7)
# init optimization model
optmodel = sudokuModel()
# init pairwise learning to rank
empty_dataset = pyepo.data.dataset.optDataset(optmodel, visual_train_sudokus[0:100], [transformToOneHot(solution).view(810) for solution in numerical_train_sudokus[0:100]])
prltr = pyepo.func.pairwiseLTR(optmodel, processes=4, solve_ratio=0.5, dataset=empty_dataset)
regret_list = []
# set the other parameters
batch_size = 8
num_epochs = 1
validation_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False)

# training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        # forward pass
        predictions = predmodel(x)
        #predictions = predmodel(x).view(x.shape[0],9,9,10)
        # Apply softmax to make probability distributions from logits
        #predictions = torch.nn.functional.softmax(predictions, dim=len(predictions.shape)-1)
        #predictions = predictions.view(x.shape[0],810)
        #loss = spop(predictions, c, w, z, reduction="mean")
        #loss = pfy(predictions, w)
        loss = prltr(predictions, c)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print("Epoch {:2},  Batch: {:2},  Loss: {:}".format(epoch+1, i+1, loss.item()))
        if i % 25 == 24:
            regret = pyepo.metric.regret(predmodel, optmodel, validation_loader)
            regret_list.append(regret)
            print("Regret: ", regret)
        # Early stopping
        # if i == 200:
        #     break


In [ ]:
# Evaluate

validationset_evaluate = CustomSudokuDatasetDFL(visual_validation_sudokus[0:500], numerical_validation_sudokus[0:500])
validation_loader_evaluate = DataLoader(validationset_evaluate, batch_size=batch_size, shuffle=False)
regret = pyepo.metric.regret(predmodel, optmodel, validation_loader_evaluate)
print("Regret: ", regret)

In [ ]:
# Manual saving

torch.save(predmodel.state_dict(), 'PieceByPieceBoardCNN10_pairwise_acc_0.9_3.pth')

## With SLN

In [ ]:
def regretForSymoblicFeedback(predmodel, sln, optmodel, dataloader):
    predmodel.eval()
    loss = 0
    optsum = 0
    # load data
    for data in dataloader:
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        # predict
        with torch.no_grad(): # no grad
            cp = predmodel(x).to("cpu").detach().numpy()
        # solve
        for j in range(cp.shape[0]):
            # SLN prediction
            visual_sudoku = x[j]
            grid_shape = visual_sudoku.shape[:2]
            # reshape from 9x9x1x28x28 to 81x1x28x28 
            pred = sln(visual_sudoku.flatten(0,1))
            #pred = cnn(visual_sudoku.unsqueeze(0).to(device))
            pred = pred.view(9,9,10)
            pred = F.softmax(pred, dim=2)
            pred = pred.cpu()  # Move the tensor to the CPU
            # our NN return 81 probabilistic vector: an 81x10 matrix
            ml_predictions =  pred.reshape(*grid_shape,10).detach() # reshape as 9x9x10 tensor for easier visualisation
            optmodel.setSLNPredictions(ml_predictions)
            
            # accumulate loss
            loss += calRegretSymbolicFeedback(optmodel, cp[j], c[j].to("cpu").detach().numpy(),
                              z[j].item())
        optsum += abs(z).sum().item()
    # turn back train mode
    predmodel.train()
    # normalized
    return loss / (optsum + 1e-7)

def calRegretSymbolicFeedback(optmodel, pred_cost, true_cost, true_obj):
    # opt sol for pred cost
    optmodel.setObj(pred_cost)
    sol, _ = optmodel.solve()
    # obj with true cost
    obj = np.dot(sol, true_cost)
    # loss
    if optmodel.modelSense == 1:
        loss = obj - true_obj
    if optmodel.modelSense == -1:
        loss = true_obj - obj
    return loss

In [ ]:
# Training with SLN

# Seed
torch.manual_seed(5896)
np.random.seed(5896)

# init prediction model
predmodel = PieceByPieceBoardCNN10(use_softmax=True).to(device)
predmodel.load_state_dict(torch.load('PieceByPieceBoardCNN10_acc_0.966.pth'))
# init SLN
sln = TinyLeNet().to(device)
sln.load_state_dict(torch.load('TinyLeNet_SLN_0.955_finetuned_for_PBPCNN10_0.966_2_SPOP.pth'))
# set optimizer
optimizer = torch.optim.Adam(predmodel.parameters(), lr=1e-7)
# init optimization model
#optmodel = sudokuModel()
optmodel = sudokuModelWithSymbolicFeedback(sln)
# init SPO+ loss
spop = pyepo.func.SPOPlus(optmodel, processes=1)
regret_list = []

batch_size = 1
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False)
num_epochs = 1

# training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        # forward pass
        predictions = predmodel(x)
        # SLN prediction
        visual_sudoku = x[0]
        grid_shape = visual_sudoku.shape[:2]
        # reshape from 9x9x1x28x28 to 81x1x28x28 
        pred = sln(visual_sudoku.flatten(0,1))
        #pred = cnn(visual_sudoku.unsqueeze(0).to(device))
        pred = pred.view(9,9,10)
        pred = F.softmax(pred, dim=2)
        pred = pred.cpu()  # Move the tensor to the CPU
        # our NN return 81 probabilistic vector: an 81x10 matrix
        ml_predictions =  pred.reshape(*grid_shape,10).detach() # reshape as 9x9x10 tensor for easier visualisation
        optmodel.setSLNPredictions(ml_predictions)
        
        loss = spop(predictions, c, w, z, reduction="mean")
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print("Epoch {:2},  Batch: {:2},  Loss: {:}".format(epoch+1, i+1, loss.item()))
        if i % 250 == 249:
            regret = regretForSymoblicFeedback(predmodel, sln, optmodel, validation_loader)
            regret_list.append(regret)
            print("Regret: ", regret)
        # if i == 99:
        #     break


In [ ]:
# Training with SLN, pairwise

# Seed
torch.manual_seed(5896)
np.random.seed(5896)

# init prediction model
predmodel = PieceByPieceBoardCNN10(use_softmax=True).to(device)
predmodel.load_state_dict(torch.load('PieceByPieceBoardCNN10_pairwise_with_SLN_acc_0.966_2.pth'))
# init SLN
sln = TinyLeNet().to(device)
sln.load_state_dict(torch.load('TinyLeNet_0.959.pth'))
# set optimizer
optimizer = torch.optim.Adam(predmodel.parameters(), lr=1e-7)
# init optimization model
optmodel = sudokuModelWithSymbolicFeedback(sln)
# init pairwise learning to rank
empty_dataset = pyepo.data.dataset.optDataset(optmodel, visual_train_sudokus[0:10], [transformToOneHot(solution).view(810) for solution in numerical_train_sudokus[0:10]])
prltr = pyepo.func.pairwiseLTR(optmodel, processes=1, solve_ratio=0.5, dataset=empty_dataset)
# init rest
regret_list = []
batch_size = 1
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False)
num_epochs = 1

# training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        # forward pass
        predictions = predmodel(x)
        # SLN prediction
        visual_sudoku = x[0]
        grid_shape = visual_sudoku.shape[:2]
        # reshape from 9x9x1x28x28 to 81x1x28x28 
        pred = sln(visual_sudoku.flatten(0,1))
        #pred = cnn(visual_sudoku.unsqueeze(0).to(device))
        pred = pred.view(9,9,10)
        pred = F.softmax(pred, dim=2)
        pred = pred.cpu()  # Move the tensor to the CPU
        # our NN return 81 probabilistic vector: an 81x10 matrix
        ml_predictions =  pred.reshape(*grid_shape,10).detach() # reshape as 9x9x10 tensor for easier visualisation
        optmodel.setSLNPredictions(ml_predictions)
        
        loss = prltr(predictions, c)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print("Epoch {:2},  Batch: {:2},  Loss: {:}".format(epoch+1, i+1, loss.item()))
        if i % 250 == 249:
            regret = regretForSymoblicFeedback(predmodel, sln, optmodel, validation_loader)
            regret_list.append(regret)
            print("Regret: ", regret)
        # if i == 99:
        #     break


In [ ]:
# Evaluate predmodel

validationset_evaluate = CustomSudokuDatasetDFL(visual_validation_sudokus[0:200], numerical_validation_sudokus[0:200])
validation_loader_evaluate = DataLoader(validationset_evaluate, batch_size=batch_size, shuffle=False)
regret = regretForSymoblicFeedback(predmodel, sln, optmodel, validation_loader_evaluate)
print("Regret: ", regret)

In [ ]:
# Manual saving

torch.save(predmodel.state_dict(), '')

## Train CNN and SLN simultaneously

In [ ]:
def regretForSymoblicFeedbackWithSLNLoss(predmodel, sln, optmodel, dataloader):
    predmodel.eval()
    loss = 0
    sln_loss = 0
    optsum = 0
    total_boards = 0
    # load data
    for data in dataloader:
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        # predict
        with torch.no_grad(): # no grad
            cp = predmodel(x).to("cpu").detach().numpy()
        # solve
        for j in range(cp.shape[0]):
            # SLN prediction
            visual_sudoku = x[j]
            grid_shape = visual_sudoku.shape[:2]
            # reshape from 9x9x1x28x28 to 81x1x28x28 
            pred = sln(visual_sudoku.flatten(0,1))
            #pred = cnn(visual_sudoku.unsqueeze(0).to(device))
            pred = pred.view(9,9,10)
            pred = F.softmax(pred, dim=2)
            pred = pred.cpu()  # Move the tensor to the CPU
            # our NN return 81 probabilistic vector: an 81x10 matrix
            sln_predictions =  pred.reshape(*grid_shape,10).detach() # reshape as 9x9x10 tensor for easier visualisation
            optmodel.setSLNPredictions(sln_predictions)
            
            # accumulate loss
            loss += calRegretSymbolicFeedback(optmodel, cp[j], c[j].to("cpu").detach().numpy(),
                              z[j].item())

            # accumulate sln loss
            differing_digit_indices = optmodel._getDifferingDigitIndices()
            sln_diff_digits_preds = torch.stack([sln_predictions[index] for index in differing_digit_indices])
            diff_digits_labels = torch.stack([w[j].view(9,9,10)[index] for index in differing_digit_indices]).cpu()
            sln_loss += sln_criterion(sln_diff_digits_preds, diff_digits_labels) 
            total_boards += cp.shape[0]
            
        optsum += abs(z).sum().item()
    # turn back train mode
    predmodel.train()
    # normalized
    return (loss / (optsum + 1e-7)), (sln_loss / total_boards)

def calRegretSymbolicFeedback(optmodel, pred_cost, true_cost, true_obj):
    # opt sol for pred cost
    optmodel.setObj(pred_cost)
    sol, _ = optmodel.solve()
    # obj with true cost
    obj = np.dot(sol, true_cost)
    # loss
    if optmodel.modelSense == 1:
        loss = obj - true_obj
    if optmodel.modelSense == -1:
        loss = true_obj - obj
    return loss

In [ ]:
# Training CNN and SLN

# Seed
torch.manual_seed(5896)
np.random.seed(5896)

# init prediction model
predmodel = PieceByPieceBoardCNN10(use_softmax=True).to(device)
predmodel.load_state_dict(torch.load('PieceByPieceBoardCNN10_acc_0.966.pth'))

# init SLN
sln = TinyLeNet().to(device)
sln.load_state_dict(torch.load('TinyLeNet_0.959.pth'))
sln_criterion = nn.CrossEntropyLoss()

# set optimizers
optimizer = torch.optim.Adam(predmodel.parameters(), lr=1e-7)
optimizer_sln = torch.optim.Adam(sln.parameters(), lr=1e-4)

# init optimization model
optmodel = sudokuModelWithSymbolicFeedback(sln)

# init SPO+ loss
spop = pyepo.func.SPOPlus(optmodel, processes=1)

# init other
regret_list = []
batch_size = 1  # Has to remain 1
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False)
num_epochs = 1

# training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        
        # forward pass CNN
        predictions = predmodel(x)
        
        # SLN prediction
        visual_sudoku = x[0]
        grid_shape = visual_sudoku.shape[:2]
        # reshape from 9x9x1x28x28 to 81x1x28x28 
        pred = sln(visual_sudoku.flatten(0,1))
        pred = pred.view(9,9,10)
        pred = F.softmax(pred, dim=2)
        pred = pred.cpu()  # Move the tensor to the CPU
        # our NN return 81 probabilistic vector: an 81x10 matrix
        sln_predictions =  pred.reshape(*grid_shape,10) # reshape as 9x9x10 tensor for easier visualisation
        optmodel.setSLNPredictions(sln_predictions)

        # Solve and compute CNN loss
        loss = spop(predictions, c, w, z, reduction="mean")
        
        # backward pass for CNN
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Fine-tune SLN using the differing digits from the solved visual sudoku
        differing_digit_indices = optmodel._getDifferingDigitIndices()
        sln_diff_digits_preds = torch.stack([sln_predictions[index] for index in differing_digit_indices])
        diff_digits_labels = torch.stack([w.view(9,9,10)[index] for index in differing_digit_indices]).cpu()
        sln_loss = sln_criterion(sln_diff_digits_preds, diff_digits_labels)
        optimizer_sln.zero_grad()
        sln_loss.backward()
        optimizer_sln.step()

        # Print progress
        print("Epoch {:2},  Batch: {:2},  Loss: {:}".format(epoch+1, i+1, loss.item()))
        if i % 250 == 249:
            regret, sln_loss = regretForSymoblicFeedbackWithSLNLoss(predmodel, sln, optmodel, validation_loader)
            regret_list.append(regret)
            print("Regret: ", regret, " SLN loss: ", sln_loss.item())

        # Break for early stopping if needed
        # if i == 99:
        #     break

In [ ]:
# Training CNN and SLN, pairwise

# Seed
torch.manual_seed(5896)
np.random.seed(5896)

# init prediction model
predmodel = PieceByPieceBoardCNN10(use_softmax=True).to(device)
predmodel.load_state_dict(torch.load('PieceByPieceBoardCNN10_pairwise_acc_0.966_simultaneous_3.pth'))

# init SLN
sln = TinyLeNet().to(device)
sln.load_state_dict(torch.load('TinyLeNet_SLN_0.957_simultaneous_3.pth'))
sln_criterion = nn.CrossEntropyLoss()

# set optimizers
optimizer = torch.optim.Adam(predmodel.parameters(), lr=1e-7)
optimizer_sln = torch.optim.Adam(sln.parameters(), lr=1e-4)

# init optimization model
optmodel = sudokuModelWithSymbolicFeedback(sln)

# init pairwise learning to rank
empty_dataset = pyepo.data.dataset.optDataset(optmodel, visual_train_sudokus[0:10], [transformToOneHot(solution).view(810) for solution in numerical_train_sudokus[0:10]])
prltr = pyepo.func.pairwiseLTR(optmodel, processes=1, solve_ratio=0.5, dataset=empty_dataset)

# init other
regret_list = []
batch_size = 1  # Has to remain 1
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False)
num_epochs = 1

# training loop
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        x, c, w, z = data
        x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
        
        # forward pass CNN
        predictions = predmodel(x)
        
        # SLN prediction
        visual_sudoku = x[0]
        grid_shape = visual_sudoku.shape[:2]
        # reshape from 9x9x1x28x28 to 81x1x28x28 
        pred = sln(visual_sudoku.flatten(0,1))
        pred = pred.view(9,9,10)
        pred = F.softmax(pred, dim=2)
        pred = pred.cpu()  # Move the tensor to the CPU
        # our NN return 81 probabilistic vector: an 81x10 matrix
        sln_predictions =  pred.reshape(*grid_shape,10) # reshape as 9x9x10 tensor for easier visualisation
        optmodel.setSLNPredictions(sln_predictions.detach())

        # Solve and compute CNN loss
        loss = prltr(predictions, c)
        
        # backward pass for CNN
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Fine-tune SLN using the differing digits from the solved visual sudoku
        differing_digit_indices = optmodel._getDifferingDigitIndices()
        sln_diff_digits_preds = torch.stack([sln_predictions[index] for index in differing_digit_indices])
        diff_digits_labels = torch.stack([w.view(9,9,10)[index] for index in differing_digit_indices]).cpu()
        sln_loss = sln_criterion(sln_diff_digits_preds, diff_digits_labels)
        optimizer_sln.zero_grad()
        sln_loss.backward()
        optimizer_sln.step()

        # Print progress
        print("Epoch {:2},  Batch: {:2},  Loss: {:}".format(epoch+1, i+1, loss.item()))
        if i % 250 == 249:
            regret, sln_loss = regretForSymoblicFeedbackWithSLNLoss(predmodel, sln, optmodel, validation_loader)
            regret_list.append(regret)
            print("Regret: ", regret, " SLN loss: ", sln_loss.item())

        # Break for early stopping if needed
        # if i == 99:
        #     break

In [ ]:
# Evaluate predmodel

validationset_evaluate = CustomSudokuDatasetDFL(visual_validation_sudokus[0:200], numerical_validation_sudokus[0:200])
validation_loader_evaluate = DataLoader(validationset_evaluate, batch_size=batch_size, shuffle=False)
regret, sln_loss = regretForSymoblicFeedbackWithSLNLoss(predmodel, sln, optmodel, validation_loader_evaluate)
print("Regret: ", regret, " SLN loss: ", sln_loss.item())

In [ ]:
# Manual saving

torch.save(predmodel.state_dict(), 'PieceByPieceBoardCNN10_SPOP.pth')
torch.save(sln.state_dict(), 'TinyLeNet_SLN_simultaneous.pth')

# Testing

In [ ]:
# Validation

# Manual loading
trained_cnn = PieceByPieceBoardCNN10()
trained_cnn.load_state_dict(torch.load('PieceByPieceBoardCNN10.pth'))
trained_cnn.eval()
#trained_cnn = predmodel
trained_sln = TinyLeNet()
trained_sln.load_state_dict(torch.load('TinyLeNet.pth'))
trained_sln.eval()

total = 0
normal_correct = 0
feedback_correct = 0
four_boards_correct = 0
normal_correct_feedback_not = 0
feedback_correct_normal_not = 0

for i in range(0, 1):
    numerical_sudoku = numerical_validation_sudokus[i]
    visual_sudoku = visual_validation_sudokus[i]

    normal_result = inference_simple(visual_sudoku, trained_cnn)["perception"]
    feedback_result = inference_with_feedback_double_cnn_simple(visual_sudoku, trained_cnn, trained_sln, 3)["perception"]
    four_boards = top_k_inference_simple(visual_sudoku, trained_cnn, k=3)

    normal_correct_bool = False
    feedback_correct_bool = False
    
    if np.array_equal(numerical_sudoku, normal_result):
        normal_correct += 1
        normal_correct_bool = True
    if np.array_equal(numerical_sudoku, feedback_result):
        feedback_correct += 1
        feedback_correct_bool = True
    for board in four_boards:
        if np.array_equal(numerical_sudoku, board["perception"]):
            four_boards_correct += 1
            break

    total += 1
    if normal_correct_bool and not feedback_correct_bool:
        normal_correct_feedback_not += 1
        print("score: ", normal_correct_feedback_not, feedback_correct_normal_not)
    elif feedback_correct_bool and not normal_correct_bool:
        feedback_correct_normal_not += 1
        print("score: ", normal_correct_feedback_not, feedback_correct_normal_not)
    print(total, normal_correct/total, feedback_correct/total, four_boards_correct/total)

    

In [ ]:
# Testing

# Manual loading
trained_cnn = PieceByPieceBoardCNN10()
trained_cnn.load_state_dict(torch.load('PieceByPieceBoardCNN10_SPOP.pth'))
trained_cnn.eval()
trained_sln = TinyLeNet()
trained_sln.load_state_dict(torch.load('TinyLeNet_SLN_simultaneous.pth'))
trained_sln.eval()

total = 0
normal_correct = 0
feedback_correct = 0
four_boards_correct = 0
normal_correct_feedback_not = 0
feedback_correct_normal_not = 0

for i in range(0, 2000):
    numerical_sudoku = numerical_test_sudokus[i]
    visual_sudoku = visual_test_sudokus[i]

    normal_result = inference_simple(visual_sudoku, trained_cnn)["perception"]
    feedback_result = inference_with_feedback_double_cnn_simple(visual_sudoku, trained_cnn, trained_sln, 3)["perception"]
    four_boards = top_k_inference_simple(visual_sudoku, trained_cnn, k=3)

    normal_correct_bool = False
    feedback_correct_bool = False
    
    if np.array_equal(numerical_sudoku, normal_result):
        normal_correct += 1
        normal_correct_bool = True
    if np.array_equal(numerical_sudoku, feedback_result):
        feedback_correct += 1
        feedback_correct_bool = True
    for board in four_boards:
        if np.array_equal(numerical_sudoku, board["perception"]):
            four_boards_correct += 1
            break

    total += 1
    if normal_correct_bool and not feedback_correct_bool:
        normal_correct_feedback_not += 1
        print("score: ", normal_correct_feedback_not, feedback_correct_normal_not)
    elif feedback_correct_bool and not normal_correct_bool:
        feedback_correct_normal_not += 1
        print("score: ", normal_correct_feedback_not, feedback_correct_normal_not)
    print(total, normal_correct/total, feedback_correct/total, four_boards_correct/total)

    

In [ ]:
# Test accuracy of CNN on digits of the visual sudoku boards

cnn = predmodel

total = 0
cnn_correct = 0
for i in range(0, 2000):
    numerical_sudoku = numerical_test_sudokus[i]
    visual_sudoku = visual_test_sudokus[i]

    ml_predictions = predict_proba_sudoku(cnn, visual_sudoku)

    _, cnn_prediction = torch.max(torch.tensor(ml_predictions).data, -1)
    cnn_prediction = cnn_prediction.numpy()
    non_zero = (numerical_sudoku != 0).sum().item()
    cnn_correct += ((cnn_prediction == numerical_sudoku.numpy())).sum().item()
    
    total += 81
    print(i, cnn_correct/total)

In [ ]:
# Test naive board accuracy

cnn = predmodel

total = 0
correct = 0
feedback_correct = 0

for i in range(0, 2000):
    numerical_sudoku = numerical_test_sudokus[i]
    visual_sudoku = visual_test_sudokus[i]
    
    predictions = predict_proba_sudoku(cnn, visual_sudoku)
    most_likely_label = np.argmax(predictions, axis=-1)

    if np.array_equal(numerical_sudoku, most_likely_label):
        correct += 1
    total +=1

    print(i, correct/total)